Install rasa_nlu[spacy], rasa_core, nest_asyncio and ipython 


In [ ]:
!pip install nest_asyncio==1.3.3

In [ ]:
!pip install rasa_nlu[spacy]  #restart runtime after installation and rerun previous cells.

In [ ]:
!pip install rasa_core

In [ ]:
pip install -U ipython #restart runtime after installation and rerun previous cells.

Import rasa_core, rasa_nlu spacy and sys



In [ ]:
import rasa_core
import rasa_nlu
import spacy

In [ ]:
import sys
python = sys.executable

download en_core_web_md, a medium-sized english model trained on written web text.

In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 120.8 MB 1.4 MB/s 

    Linking successful
    /usr/local/lib/python3.7/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.7/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



Declearation of intent and sample words/phrases/sentences which refer to the intention decleared, which will be stored inside nlu.md. In other words, these are sample phrases whcih the chatbot will learn from for each intent.

In [ ]:
# Writing various intents with examples to nlu.md file

nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad

## intent:bot_challenge
- are you a bot?
- are you a human?
- am I talking to a bot?
- am I talking to a human?

## intent:movie_match_year
- what were the movies made in 2018?
- what were the movies made in 1980?
- what were the movies made in 1995?
- what were the movies made in 1990?
- what were the movies made in 2005?
- what movies have been released in 2018?
- what movies have been released in 1990?
- what movies have been released in 2000?

## synonym:movies
- films

## regex:rating
- top \b[0-9]+\b

##synonym:made
- released
- filmed
- produced
- created
- formed

##intent:get_year_by_movie_title
- when was released [the godfather](movie_title)?
- when was released [star wars](movie_title)?
- when was released [fight club](movie_title)?
- when was released [interstellar](movie_title)?
- when was released [inception](movie_title)?
- what year did [miss jerry](movie_title) come out?

##lookup:movie_title
- the shawshank redemption
- the dark knight
- inception
- fight club
- pulp fiction
- forrest gump
- the matrix
- the lord of the rings: the fellowship of the ring
- the lord of the rings: the return of the king
- the godfather
- the dark knight rises
- the lord of the rings: the two towers
- interstellar
- se7en
- gladiator
- django unchained
- the avengers
- batman begins
- the silence of the lambs
- inglourious basterds
- star wars
- saving private ryan
- schindler's list
- the departed
- the prestige
- star wars: episode v - the empire strikes back

##intent:movie_match_several_criteria
- do you have any movie from last year with [awesome special effect](movie_attribute)?
- do you have any movie from two years ago with [awesome special effect](movie_attribute)?
- do you have any movie from three years ago with [awesome special effect](movie_attribute)?
- tell me [drama](genre) movie that [people love for the love story](movie_attribute)
- tell me movies that have [incredible landscapes and nature scenes](movie_attribute)
- what films do you have that include [space travel and aliens](movie_attribute)?
- tell me movies that have [awesome special effect](movie_attribute)
- tell me movies that have [no special effect](movie_attribute)
- tell me movies that have [cats and dogs](movie_attribute)

##lookup:movie_attribute
- awesome special effect
- no special effect
- people love for the love story
- incredible landscapes and nature scenes
- space travel and aliens
- cats and dogs
- a lot of blood and war
- super hero
- based on a true story
"""


%store nlu_md >nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


Setting up our configuration for our pipeline and policies

In [ ]:
# Adding the NLU components to the pipeline in config.yml file
config = """
language: "en_core_web_md"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                     # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms

policies:
- name: "RulePolicy"
  # Confidence threshold for the `core_fallback_action_name` to apply.
  # The action will apply if no other action was predicted with
  # a confidence >= core_fallback_threshold
  core_fallback_threshold: 0.68
  core_fallback_action_name: "action_default_fallback"
  enable_fallback_prediction: True


""" 

%store config >config.yml

Writing 'config' (str) to file 'config.yml'.


# Training the model
Import the load_data and config modules so as to load the training data file and config file respectiely, which would be fed to the Trainer module to train the model.

In [ ]:
# Import modules for training
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")
trainer = Trainer(config.load("config.yml"))

# training the nlu
interpreter = trainer.train(training_data)
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

/usr/local/lib/python3.7/dist-packages/rasa_nlu/training_data/training_data.py:184: UserWarning: Entity 'genre' has only 1 training examples! minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_ENTITY))


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

Testing our model.
The aim is to see how well it can predict intents, this would be then use to determind the appropriate response.

In [ ]:
 # Testing the NLU model with an input message

import json
def pprint(o):   
    print(json.dumps(o, indent=2))

pprint(interpreter.parse("x"))

{
  "intent": {
    "name": "get_rating_by_movie_title",
    "confidence": 0.45100947442706885
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "get_rating_by_movie_title",
      "confidence": 0.45100947442706885
    },
    {
      "name": "bot_challenge",
      "confidence": 0.12219982193006196
    },
    {
      "name": "movie_match_several_criteria",
      "confidence": 0.08845532252536374
    },
    {
      "name": "get_year_by_movie_title",
      "confidence": 0.07743052070803512
    },
    {
      "name": "movie_match_year",
      "confidence": 0.0696327189968297
    },
    {
      "name": "mood_great",
      "confidence": 0.046400707897303196
    },
    {
      "name": "get_genre_by_movie_title",
      "confidence": 0.04519555946848949
    },
    {
      "name": "goodbye",
      "confidence": 0.03736394018575891
    },
    {
      "name": "affirm",
      "confidence": 0.019958409984586353
    },
    {
      "name": "greet",
      "confidence": 0.01810636798307734

This is the second part of the project where we would be dealing with stories, actions and templates so our bot can respond to queries appropriately. Stories is a path which countains intents and their respective actions to be taken.
Actions are the the responses we want our bot to give after identifying intent.
Templates are where we store all the actions.
Slots are our bots memories which help it keep track of events.

In [ ]:
# Writing stories and saving it in the stories.md file

stories_md = """
## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

## bot challenge
* bot_challenge
  - utter_iamabot

## search movie by title
* movie_match_title
    - slot{"title": "Avengers"}
    - action_match_title_search_movie    

## search movie by year
* movie_match_year
    - slot{"time": "1995"}
    - action_match_year_search_movie

## search year with movie title
* get_year_by_movie_title
   - slot{"movie_title": "joker"}
   - action_get_year_by_movie_title
   
## search rating with movie title
* get_rating_by_movie_title
   - slot{"movie_title": "joker"}
   - action_get_rating_by_movie_title
   
   
##search for movies based on its attributes
* movie_match_several_criteria
    - slot{"movie_attribute": "cats and dogs"}
    - action_get_movie_based_attribute

"""

%store stories_md >stories.md

Writing 'stories_md' (str) to file 'stories.md'.


The domain is where we link together all our files for the chatbot. The uttar_default action is he action to be taken when the confidence value of all possible intentions is below the thereshold value set inn our configuration file under the fallback policy.

In [ ]:
# Writing all the intents,slots,entities,actions and templates to domain.yml

domain_yml = """
intents:
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - bot_challenge
  - movie_match_title
  - movie_match_year
  - movie_match_several_criteria
  - get_year_by_movie_title
  - get_rating_by_movie_title
entities:
- rating
- movie_title
- time
- movie_attribute
slots:
  movie_attribute:
    type: text
  movie_title:
    type: text
  rating:
    type: text
  time:
    type: text
templates:
  utter_greet:
    - text: Hey! How are you?
  utter_cheer_up:
    - text: 'Here is something to cheer you up:'
      image: https://i.imgur.com/nGF1K8f.jpg
  utter_did_that_help:
    - text: Did that help you?
  utter_happy:
    - text: Great, carry on!
  utter_goodbye:
    - text: Bye
  utter_iamabot:
    - text: I am a bot, powered by Rasa.
  utter_default:
    - text: sorry, I didn't get that, can you rephrase it?
  utter_ask_time:
    - text: What is the year?
  utter_ask_rating:
    - text: What is the rating?
  utter_ask_movie_title:
    - text: What is the title of the movie?
  utter_ask_movie_attribute:
    - text: What is the attributes of the movie?
  utter_movie_match_title_result:
    - text: 'Finding movies of this Title: {movie_title}'
  utter_movie_match_year_result:
    - text: 'Finding movies of this Year: {time}'
  utter_movie_match_attr_result:
    - text: 'Finding movies with the Attributes: {movie_attribute}'
actions:
  - utter_greet
  - utter_cheer_up
  - utter_did_that_help
  - utter_happy
  - utter_goodbye
  - utter_iamabot
  - utter_default
  - utter_ask_time
  - utter_ask_rating
  - utter_ask_movie_title
  - utter_ask_movie_attribute
  - utter_movie_match_title_result  
  - utter_movie_match_year_result
  - action_match_title_search_movie 
  - action_match_year_search_movie
  - action_get_year_by_movie_title
  - action_get_rating_by_movie_title
  - action_get_movie_based_attribute
 
"""

%store domain_yml >domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


Here we import our policies and the Agent module. The Agent module is used to load the file, and passed to the 'agent' model. The 'agent' model is in turn trained with the data in the stories file.

In [ ]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display
import urllib.parse

import requests

class ActionTitleMovieSearch(Action):
    def name(self):
        return "action_match_title_search_movie"

    def run(self, dispatcher, tracker, domain):
        
        title = tracker.get_slot("movie_title")
        safe_string = urllib.parse.quote_plus(title)
        r = requests.get("https://api.themoviedb.org/3/movie/550?api_key=ca8bdd9ef366d6cafb5ead8425e5efd4&language=en-US&query={}&page=1&include_adult=false".format(safe_string))
        res = r.json()
        dispatcher.utter_message(response = "utter_movie_match_title_result", movie_title = title)
        dispatcher.utter_message(res)

class ActionYearMovieSearch(Action):
    def name(self):
        return "action_match_year_search_movie"

    def run(self, dispatcher, tracker, domain):
        
        year = tracker.get_slot("time")
        r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=ca8bdd9ef366d6cafb5ead8425e5efd4&language=en-US&query=a&page=1&include_adult=false&year={}".format(str(year)))
        res = r.json()
        dispatcher.utter_message(response = "utter_movie_match_year_result", time = year)
        dispatcher.utter_message(res)

class ActionYearTitleSearch(Action):
    def name(self):
        return "action_get_year_by_movie_title"

    def run(self, dispatcher, tracker, domain):
        
        title = tracker.get_slot("movie_title")
        safe_string = urllib.parse.quote_plus(title)
        r = requests.get("https://api.themoviedb.org/3/movie/550?api_key=ca8bdd9ef366d6cafb5ead8425e5efd4&language=en-US&query={}&page=1&include_adult=false".format(safe_string))
        res = r.json()
        dispatcher.utter_message(res["results"][0]["release_date"])

class ActionRatingTitleSearch(Action):
    def name(self):
        return "action_get_rating_by_movie_title"

    def run(self, dispatcher, tracker, domain):
        
        title = tracker.get_slot("movie_title")
        safe_string = urllib.parse.quote_plus(title)
        r = requests.get("https://api.themoviedb.org/3/movie/550?api_key=ca8bdd9ef366d6cafb5ead8425e5efd4&language=en-US&query={}&page=1&include_adult=false".format(safe_string))
        res = r.json()
        dispatcher.utter_message(res["results"][0]["vote_average"])

class ActionAttrMovieSearch(Action):
    def name(self):
        return "action_get_movie_based_attribute"

    def run(self, dispatcher, tracker, domain):
        
        attr = tracker.get_slot("movie_attribute")
        safe_string = urllib.parse.quote_plus(attr)
        r = requests.get("https://api.themoviedb.org/3/movie/550?api_key=ca8bdd9ef366d6cafb5ead8425e5efd4&language=en-US&query={}&page=1&include_adult=false".format(safe_string))
        res = r.json()
        dispatcher.utter_message(response = "utter_movie_match_title_result", movie_attribute = attr)
        dispatcher.utter_message(res)

In [ ]:
# Import the policies and agent
from rasa_core.policies import FallbackPolicy, MemoizationPolicy,KerasPolicy
from rasa_core.agent import Agent

# Initialize the model with `domain.yml`
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), ])

# loading our  training dialogues from `stories.md` 
training_data = agent.load_data('stories.md')


# Training the model
agent.train(
    training_data)
    #validation_split=0.0,
    #epochs=200)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 10/10 [00:00<00:00, 116.80it/s, # trackers=17]
Processed actions: 26it [00:00, 335.35it/s, # examples=26]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 46)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                10112     
_________________________________________________________________
dense (Dense)                (None, 26)                858       
_________________________________________________________________
activation (Activation)      (None, 26)                0         
Total params: 10,970
Trainable params: 10,970
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
463/463 [==============================] - 1s 1ms/sample - loss: 3.1238 - acc: 0.2592
Epoch 2/100
463/463 [==============================] - 0s 194us/sample - loss: 2.8663 - acc: 0.4622
Epoch 3/100
463/463 [==============================] - 0s 191us/sample -

Install version 2.10.0 of h5py package

In [ ]:
!pip uninstall h5py
!pip install h5py==2.10.0

Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-2.10.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-2.10.0
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)


Load the trained model

In [ ]:
#Starting the Bot
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

Write a function to tale inputs for the chatbot and gives out an output while stopping when 'stop' is typed in.

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'" + "\n\n" + "You:")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'

You:
what were the movies made in 2018?
Bye
are you a human?
I am a bot, powered by Rasa.
what movies have been released in 2018?


KeyboardInterrupt: ignored